In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-03-26 21:59:45--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'input.txt.1'

input.txt.1         100%[===================>]   1.06M  3.21MB/s    in 0.3s    

2023-03-26 21:59:46 (3.21 MB/s) - 'input.txt.1' saved [1115394/1115394]



In [2]:
#read and inspect
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
#first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
#all the unique characters occurring in the text
chars = sorted(list(set(text))) #set of unique characters in text and print into list
vocab_size = len(chars) #number of elements in dataset
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


tokenize input text: convert raw text as string to sequence of integers
- build character level language model (translating individual characters into integers)


# Encoder and Decoder

In [6]:
# create a mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] #take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) #take a list of integers, output a string

print(encode("hii there")) # receive list of integers that represent the string
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [9]:
#encode entire text dataset and store into torch.Tensor from the shakespeare script
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

### train test split into validation sets

In [10]:
n = int(0.9*len(data)) #first 90% are train, rest val 
train_data = data[:n]
val_data = data[n:]

In [13]:
block_size = 8
train_data[:block_size+1] #sequence need to follow one another

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [15]:
x = train_data[:block_size] #input char
y = train_data[1:block_size+1] #next offset char as y is the target for each position of the input
for t in range(block_size): #iterate through blocksize of 8
    context = x[:t+1] #all characters in x up to t and including t
    target = y[t] #target is the t'th character
    print(f"when input is {context} the target: {target}") #transformer can predict every character +1

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [22]:
torch.manual_seed(1337)
batch_size = 4 #how many independent sequences will we process in parallel?
block_size = 8 #what is max context length for predictiions?

def get_batch(split):
    #generate a small batch of data of inputs x and targets y 
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) #generate batch size number of random offsets where ix will be four numbers
    x = torch.stack([data[i:i+block_size] for i in ix]) #first blocksize characters // chunk for every 1 in ix
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) #offset by 1 target > stack as row of 4x8 tensors
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb) # chunks of the training set
print('targets:')
print(yb.shape)
print(yb) #this gives the correct answer for every decision 

print('----')

for b in range(batch_size): #batch dimension 
    for t in range(block_size): #time dimension
        context = xb[b, :t+1]
        target =yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")
        
        
"""
when the input is 24, 43, 58 > target is 5
when input is 44, 53 > target is 56 
sequence in the X matrix will correspond with the position in the Y matrix
"""

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

'\nwhen the input is 24, 43, 58 > target is 5\nwhen input is 44, 53 > target is 56 \nsequence in the X matrix will correspond with the position in the Y matrix\n'

In [17]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [34]:
import torch 
import torch.nn as nn 
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        #each token reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) 
        
    def forward(self, idx, targets=None): 
        #idx and targets are both (B, T) tensor of integers 
        logits = self.token_embedding_table(idx) #(B,T,C) #index passed into token embedding table
        #loss function 
    
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C) #preserve channel dimension as own dimension
            targets = targets.view(B*T) #one dimensional 
            loss = F.cross_entropy(logits, targets) #measures the quality of logits to the targets > how well are we predicting the next character based on logit
        return logits, loss
    #every single integer is going to block out the corresponding index row
    #24 would pluck out the 24th row 
    #arrange by batch, time, channel > predict what come next just by the individual identity of a single token > logits are scores
    # BTC : B predictions, T scores, C logits for every 4x8 positions
    # the predicted number should have higher number of logits while others have low dimension
    #if have targets, provide in the loss, if no targets get from logits
    
    def generate(self, idx, max_new_tokens):
        
        #idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self(idx) #take current indices and get predictions, loss ignored as no truth targets
            
            #focus onl on the last time step 
            logits = logits[:, -1, :] #becomes (B, C) block out the last ele in time dimension to get logit
            
            #apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # (B,C)
            
            #sample from the distribution 
            #ask for 1 sample as each batch dimension will return one prediction
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            #append sampled index to the running sequence
            #integers concatenated on current running stream of ints giving up B, T+1
            idx = torch.cat((idx, idx_next), dim=1) #(B, T+1)
        return idx
    
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb) #passing the inputs and targets
print(logits.shape)
print(loss)

 #1x1 integer, gives 1 dimensional array of our indices + convert to list
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist())) 

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [35]:
#create a pytorch optimiser
optimiser = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [39]:
batch_size = 32
for steps in range(10000):
    
    #sample a batch of data
    xb, yb = get_batch('train')
    
    #evaluate the loss
    logits, loss = m(xb,yb) #printing out all the loss
    optimiser.zero_grad(set_to_none=True) #zeroing all the gradients from the prev step
    loss.backward() #get gradients for all parameters
    optimiser.step() #use gradients to update parameters
    
print(loss.item())

2.429983139038086


In [42]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=200)[0].tolist())) 


An.
DUClispe titunty sts ws.
WIUCareand.
Whtibamang I g in we ot Be, p an, be rst
THEYod indes'se angmmacomard towond pe l d aca thely matht wese' f wey torme:
APr by ine h isththa vig t quprofais shi


### tokens not talking to each other but only follow a sequence
### now will talk following context so they can predict what can become next

## Mathematical Trick in Self Attention

In [55]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
#a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

#first row of a DOT PRODUCT with first column of b  (to get matrix c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [54]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [49]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C) #5th token should only speak to previous to predict future
x.shape #5th token communicate only 4,3,2,1 tokens to current timestamp. find average of preceding tokens

#calculate average of current token and previous tokens as feature vector

torch.Size([4, 8, 2])

In [50]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B): #iterate over batch
    for t in range(T): #iterate over time 
        xprev = x[b,:t+1] # (t,C) t from the past, C is 2dimensional info from the tokens
        xbow[b,t] = torch.mean(xprev, 0) #averaging out the t, and get 1 dimensional vector

In [56]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T)) #weights are speciied by the TxT array. we're doing weighted sums. 
#Ensures it will only get info from tokens preceding it
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C) xbow2 will be the same as xbow
torch.allclose(xbow, xbow2)

True

In [59]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [60]:
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) #every 0's will be inf. 
#setting to negative inf, we will not aggregate anything from the tokens
#inf between the 0's will become data dependent + start looking at each other to find other tokens
#depending on values, they will find each other interesting to diff amounts. 
#++ the future cant communicate w the past + find weighted aggregation from past elements
#wei = F.softmax(wei, dim=1) #normalisation > exponentiate and divide by sum
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [57]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x 
#matrix multiplication of lower triangular fashion + lower elements of triangle will tell how much each element fuses into this position
torch.allclose(xbow, xbow3)


True

In [47]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)
#every single token will emit two vector: QUERY (looking for) + KEY (what do i contain)
#affinities: Query DOT PRODUCT with Key = wei
# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x) #propagate the liniear
out = wei @ v #v is the element we aggregated
#out = wei @ x

out.shape

#vowel and looking for consonant. try look for specific information from the past and do it in data dependent way

torch.Size([4, 8, 16])

In [48]:
wei[0] #im a vowel, need consonants. im in the 8th position +look for pos up to 4. creates a query
#All the consonants of positions up to 4. all those that satisfies the criteria, make the key have a high number of affinity
#when query + key backpropagate, they create high affinity
#through softmax, we get to learn alot about its position

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:

- Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

- every node has a vector of info + gets to aggregate info via a weighted sum from all the nodes that point to it
- done in a data dependent manner. depending on data 
- 8 nodes bc block size is 8. theres always 8 tokens.
- first node points to itself. 8th node is all prev nodes  itself
- attention is a set of vectors in graph. by default, the node dont know where they are in the space
- need to encode node to position to let them kmow where they are
- convolution has specific layout in space + convolutional filters act in space 
- speicfically add to space relative to positional encodings
- always processed independently via matrix multiplication 
- four seperate pools of 8 nodes
- transformers: want all nodes to talk to each other fully 
- need to use encoder block for self attention in the -inf block
- nodes from the future never talk to the pat
- wei takes very positive or negative numbers, softmax will converge towards 1 1/2 vectors

## MULTI HEAD ATTENTION

In [67]:
class BatchNorm1d:
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x-xmean) /torch.sqrt(xvar + self.eps) #normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # update the buffers
    return self.out

    def parameters(self):
        return [self.gamma, self.beta]

    torch.manual_seed(1337)
    module = BatchNorm1d(100)
    x = torch.randn(32, 100) #batch size 32 of 100 dimensional vectors
    x = module(x)
    x.shape

In [68]:
x[:,0].mean(), x[:,0].std() #mean, std of one feature across all batch inputs

(tensor(0.1560), tensor(0.8894))

In [69]:
x[0,:].mean(), x[0,:].std() #mean, std of single input from the batch, of its features

(tensor(0.0496), tensor(0.7829))